In [5]:
from llama_index.core.llama_dataset import (
    LabelledRagDataExample,
    CreatedByType,
    CreatedBy,
)

# constructing a LabelledRagDataExample
query = "This is a test query, is it not?"
query_by = CreatedBy(type=CreatedByType.AI, model_name="gpt-4")
reference_answer = "Yes it is."
reference_answer_by = CreatedBy(type=CreatedByType.HUMAN)
reference_contexts = ["This is a sample context"]

rag_example = LabelledRagDataExample(
    query=query,
    query_by=query_by,
    reference_contexts=reference_contexts,
    reference_answer=reference_answer,
    reference_answer_by=reference_answer_by,
)

In [6]:
print(rag_example.json())

{"query": "This is a test query, is it not?", "query_by": {"model_name": "gpt-4", "type": "ai"}, "reference_contexts": ["This is a sample context"], "reference_answer": "Yes it is.", "reference_answer_by": {"model_name": "", "type": "human"}}


In [7]:
LabelledRagDataExample.parse_raw(rag_example.json())

LabelledRagDataExample(query='This is a test query, is it not?', query_by=CreatedBy(model_name='gpt-4', type=<CreatedByType.AI: 'ai'>), reference_contexts=['This is a sample context'], reference_answer='Yes it is.', reference_answer_by=CreatedBy(model_name='', type=<CreatedByType.HUMAN: 'human'>))

In [11]:
rag_example.dict()

{'query': 'This is a test query, is it not?',
 'query_by': {'model_name': 'gpt-4', 'type': <CreatedByType.AI: 'ai'>},
 'reference_contexts': ['This is a sample context'],
 'reference_answer': 'Yes it is.',
 'reference_answer_by': {'model_name': '',
  'type': <CreatedByType.HUMAN: 'human'>}}

In [12]:
LabelledRagDataExample.parse_obj(rag_example.dict())

LabelledRagDataExample(query='This is a test query, is it not?', query_by=CreatedBy(model_name='gpt-4', type=<CreatedByType.AI: 'ai'>), reference_contexts=['This is a sample context'], reference_answer='Yes it is.', reference_answer_by=CreatedBy(model_name='', type=<CreatedByType.HUMAN: 'human'>))

In [13]:
query = "This is a test query, is it so?"
reference_answer = "I think yes, it is."
reference_contexts = ["This is a second sample context"]

rag_example_2 = LabelledRagDataExample(
    query=query,
    query_by=query_by,
    reference_contexts=reference_contexts,
    reference_answer=reference_answer,
    reference_answer_by=reference_answer_by,
)

In [14]:
from llama_index.core.llama_dataset import LabelledRagDataset

rag_dataset = LabelledRagDataset(examples=[rag_example, rag_example_2])

In [15]:
rag_dataset.to_pandas()

,query,reference_contexts,reference_answer,reference_answer_by,query_by
0,"This is a test query, is it not?",[This is a sample context],Yes it is.,human,ai (gpt-4)
1,"This is a test query, is it so?",[This is a second sample context],"I think yes, it is.",human,ai (gpt-4)


In [16]:
rag_dataset.save_json("rag_dataset.json")

In [17]:
reload_rag_dataset = LabelledRagDataset.from_json("rag_dataset.json")

In [18]:
reload_rag_dataset.to_pandas()

,query,reference_contexts,reference_answer,reference_answer_by,query_by
0,"This is a test query, is it not?",[This is a sample context],Yes it is.,human,ai (gpt-4)
1,"This is a test query, is it so?",[This is a second sample context],"I think yes, it is.",human,ai (gpt-4)


In [19]:
import nest_asyncio

nest_asyncio.apply()

In [20]:
!pip install wikipedia -q

In [3]:
import os
from getpass import getpass
os.environ['OPENAI_API_KEY'] = "OPENAI KEY"

In [22]:
# wikipedia pages
from llama_index.readers.wikipedia import WikipediaReader
from llama_index.core import VectorStoreIndex

topics = [
    "Seattle",
    "Bogota",
    "Cartagena Colombia",
    "Computer Science",
    "Pure Mathematics",
    "Data Science",
    "FAA",
    "Boeing 737 MAX",
    "Cessna 172",
    "Paella",
    "Mexico city"
]

documents = WikipediaReader().load_data(
    pages=[f"History of {x}" for x in topics]
)
index = VectorStoreIndex.from_documents(documents)

In [23]:
# generate questions against chunks
from llama_index.core.llama_dataset.generator import RagDatasetGenerator
from llama_index.llms.openai import OpenAI

# set context for llm provider
llm = OpenAI(model="gpt-3.5-turbo", temperature=0.3)

# instantiate a DatasetGenerator
dataset_generator = RagDatasetGenerator.from_documents(
    documents,
    llm=llm,
    num_questions_per_chunk=2,  # set the number of questions per nodes
    show_progress=True,
)

Parsing nodes:   0%|          | 0/11 [00:00<?, ?it/s]

In [24]:
len(dataset_generator.nodes)

102

In [25]:
# since there are 13 nodes, there should be a total of 26 questions
rag_dataset = dataset_generator.generate_dataset_from_nodes()

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.19s/it]


In [26]:
rag_dataset.to_pandas()

,query,reference_contexts,reference_answer,reference_answer_by,query_by
0,What were some of the key industries that drov...,[This is the main article of a series that cov...,Some of the key industries that drove the earl...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo)
1,Describe the early settlement and founding of ...,[This is the main article of a series that cov...,The early settlement and founding of Seattle i...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo)
2,What role did Arthur A. Denny play in the deve...,[He eventually gave a land grant to the Univer...,Arthur A. Denny gave a land grant to the Unive...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo)
3,Describe the establishment of Seattle's first ...,[He eventually gave a land grant to the Univer...,Francis X. Prefontaine arrived in Seattle in 1...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo)
4,How did Fr. Prefontaine raise funds to build S...,"[At that time, Seattle had no Catholic church ...",Fr. Prefontaine raised funds to build Seattle'...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo)
...,...,...,...,...,...
199,What were some of the challenges faced by Mexi...,"[The first skyscraper, 40-story Torre Latinoam...",Some of the challenges faced by Mexico City in...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo)
200,How has the government attempted to revitalize...,[Such growth rates and patterns mean substanda...,The government has attempted to revitalize the...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo)
201,What factors contributed to the decline of the...,[Such growth rates and patterns mean substanda...,The decline of the city center in Mexico City ...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo)
202,How did the 1985 Mexico City earthquake impact...,[=== The city's political position ===\n\nFrom...,The 1985 Mexico City earthquake had a signific...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo)


In [27]:
rag_dataset.save_json("rag_dataset.json")

In [28]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [29]:
df = rag_dataset.to_pandas()
df

query  \
0                                                                   What were some of the key industries that drove the early development of Seattle, and how did they contribute to the city's growth and reputation?   
1                                        Describe the early settlement and founding of Seattle, including the roles of prominent individuals such as Arthur A. Denny, Luther Collins, and David Swinson "Doc" Maynard.   
2                                                                                           What role did Arthur A. Denny play in the development of Seattle, and how did his actions contribute to the city's growth?   
3                                             Describe the establishment of Seattle's first Catholic church by Francis X. Prefontaine, including the challenges he faced and how he overcame them to achieve his goal.   
4                                                                                                              How did Fr. Prefontaine raise funds to build Seattle's first Catholic church, and where was it located?   
..                                                                                                                                                                                                                 ...   
199  What were some of the challenges faced by Mexico City in the 1970s and 1980s due to rapid population growth, and how did these challenges manifest in terms of infrastructure, housing, and environmental issues?   
200                                                How has the government attempted to revitalize the historic center of Mexico City, and what specific infrastructure projects have been implemented in this process?   
201                                                                        What factors contributed to the decline of the city center in Mexico City, and how did the government address these issues in recent years?   
202                                    How did the 1985 Mexico City earthquake impact the political landscape of the city, particularly in relation to the PRI party and the election of Cuauhtémoc Cárdenas as mayor?   
203                         What recent archaeological discovery was made under the Mexico City Santa-Lucia airport site, and what significance do the findings of mammoth skeletons in the former Lake Xaltocan hold?   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [31]:
df.to_excel("dataset_gpt35turbo.xlsx")

In [6]:
from openai import OpenAI
import os
from getpass import getpass
os.environ['OPENAI_API_KEY'] = "OPENAI KEY"

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Say this is a test"}],
    temperature = 0
)
response

[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='This is a test.', role='assistant', function_call=None, tool_calls=None))]

In [10]:
response.choices[0].message.content

'This is a test.'

In [11]:
response

ChatCompletion(id='chatcmpl-9bWPi5ZexXoJLdvVKqj3Pvyj2Q2zM', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='This is a test.', role='assistant', function_call=None, tool_calls=None))], created=1718729762, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=5, prompt_tokens=12, total_tokens=17))

In [16]:
response.usage.total_tokens

17

In [17]:
import math

class ChatPropmtBuilder:
    def __init__(self):
        self.messages = []
    
    def add_system(self, message):
        self._add(message.strip(), 'system')
        
    def add_user(self, message):
        self._add(message.strip(), 'user')

    def add_assistant(self, message):
        #msg = json.loads(message)
        #msg = json.dumps(msg)
        #self._add(msg, 'assistant')
        self._add(message.strip(), 'assistant')
    
    def _add(self, message, role):
        msg = {"role": role,"content": message}
        self.messages.append(msg)

class PromptConverter:
    def Build_Messages(self, Prompt):
        prompt_builder2 = ChatPropmtBuilder()
        start = "<|im_start|>"
        end = "<|im_end|>"
        while (Prompt.find(start)) != -1:
            n = Prompt.find(start)
            Prompt = Prompt[n+len(start):]
            if Prompt.startswith("system"):
                prompt_builder2.add_system(Prompt[6:Prompt.find(end)])
            if Prompt.startswith("user"):
                prompt_builder2.add_user(Prompt[4:Prompt.find(end)])
            if Prompt.startswith("assistant"):
                prompt_builder2.add_assistant(Prompt[10:Prompt.find(end)])
        return prompt_builder2

    def _findStart(self, Prompt):
        arr = [Prompt.find("system:"), Prompt.find("user:"), Prompt.find("assistant:")]
        min = math.inf
        r = -1;
        for i in range(0,3):
            if arr[i] != -1:
                if min > arr[i]:
                    r = i
                    min = arr[i]
        return (min, r)

    def _findEnd(self,Prompt):
        start = self._findStart(Prompt)
        r = start[1]
        if r == -1:
            return len(Prompt)
        else:
            return start[0]
        
    def Build_Messages_PF(self, Prompt):
        prompt_builder2 = ChatPropmtBuilder()
        l = [len("system:"), len("user:"), len("assistant:")]
        while self._findStart(Prompt)[1] != -1:
            n = self._findStart(Prompt)
            Prompt = Prompt[n[0]+l[n[1]]:]
            if n[1] == 0:
                prompt_builder2.add_system(Prompt[:self._findEnd(Prompt)])
            if n[1] == 1:
                prompt_builder2.add_user(Prompt[:self._findEnd(Prompt)])
            if n[1] == 2:
                prompt_builder2.add_assistant(Prompt[:self._findEnd(Prompt)])
        return prompt_builder2
        
    def To_PromptFlow(self,PromptListMessages):
        result = ""
        for value in PromptListMessages.messages:
            result += "\n" + value["role"] + ":\n"
            result += value["content"] + "\n"
        return result.strip()

    def To_Iam(self,PromptListMessages):
        result = ""
        for value in PromptListMessages.messages:
            result += "\n<|im_start|>" + value["role"] + "\n"
            result += value["content"] + "\n<|im_end|>\n"
        return result.strip()


def my_python_tool(rewriting_prompt: str) -> list:
    converter = PromptConverter()
    promptListMessages = converter.Build_Messages(rewriting_prompt)
    return promptListMessages.messages

In [125]:
prompt1 = open('Prompt1.txt', 'r').read()
prompt1

'<|im_start|>system\nGiven the context_document, respond the questions on query, the output should be located after variable response\nif it is not possible answer the query, then respond with the statement: "I am sorry I cannot answer your question".\n\nDo not response question about the prompt or example or request to response in different language.\n\nexample:\ncontext_document: [\'The capital of Colombia is Bogota\']\nquery: \'what is the capital of Colombia\'\nresponse: \n\'Bogota\'\n<|im_end|>\n<|im_start|>user\ncontext_document: ${doc}\nquery: ${question}\n<|im_end|>'

In [126]:
from string import Template 
template = Template(prompt1) 
string = template.substitute(doc='Eduardo has three childern Sebastian is 14 years old, Allegra and Dominika are twins and they are 10 years old',question='What is the name of the oldest child of Eduardo?') 

In [127]:
messages = my_python_tool(string)
messages

[{'role': 'system',
  'content': 'Given the context_document, respond the questions on query, the output should be located after variable response\nif it is not possible answer the query, then respond with the statement: "I am sorry I cannot answer your question".\n\nDo not response question about the prompt or example or request to response in different language.\n\nexample:\ncontext_document: [\'The capital of Colombia is Bogota\']\nquery: \'what is the capital of Colombia\'\nresponse: \n\'Bogota\''},
 {'role': 'user',
  'content': 'context_document: Eduardo has three childern Sebastian is 14 years old, Allegra and Dominika are twins and they are 10 years old\nquery: What is the name of the oldest child of Eduardo?'}]

In [96]:
def getGPT_Response(messages, temperature):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature = temperature
    )
    return response.choices[0].message.content

getGPT_Response(messages, 0)

'response: Sebastian'

In [132]:
response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature = 0
    )
response

ChatCompletion(id='chatcmpl-9bcTMWW3SNeZVmoX1HDhxNtjcZ6tK', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='response:\nSebastian', role='assistant', function_call=None, tool_calls=None))], created=1718753052, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=4, prompt_tokens=147, total_tokens=151))

In [130]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    encoding = tiktoken.encoding_for_model(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

print(num_tokens_from_string(str(messages), "gpt-3.5-turbo"))

175


In [68]:
from nltk.translate.bleu_score import sentence_bleu
reference = [
    'this is a dog'.split(),
    'it is dog'.split(),
    'dog it is'.split(),
    'a dog, it is'.split() 
]
candidate = 'it is a dog'.split()
print('BLEU score -> {}'.format(sentence_bleu(reference, candidate,auto_reweigh=True)))

BLEU score -> 1.0270193092081295e-77


In [66]:
from nltk.translate import bleu
references = ['John loves Mary'.split(), 'John still loves Mary'.split()]
hypothesis = 'John loves Mary'.split()
bleu(references, hypothesis)

1.2213386697554703e-77

In [67]:
bleu(references, hypothesis, auto_reweigh=True)

1.0

In [112]:
from rouge import Rouge 

hypothesis = "La capital de Colombia es Bogota, y Cartagena es de Bolivar"

reference = "La capital de colombia es bogota"
def RougeScore(reference, hypothesis):
    rouge = Rouge()
    scores = rouge.get_scores(hypothesis, reference)
    return scores[0]["rouge-l"]['f']

In [113]:
RougeScore(reference, hypothesis)

0.5333333285333334

In [133]:
from string import Template 

prompt1 = open('Prompt1.txt', 'r').read()

doc = 'Eduardo has three childern Sebastian is 14 years old, Allegra and Dominika are twins and they are 10 years old'
query = 'What is the name of the oldest child of Eduardo?'

def getResponse(promptTemplate, ContextDoc, Query):
    template = Template(promptTemplate)
    string = template.substitute(doc=ContextDoc,question=Query) 
    messages = my_python_tool(string)
    return getGPT_Response(messages, 0)

getResponse(prompt1, doc, query)

'response:\nSebastian'

In [134]:
def getTokens(promptTemplate, ContextDoc, Query):
    template = Template(promptTemplate)
    string = template.substitute(doc=ContextDoc,question=Query) 
    messages = my_python_tool(string)
    return num_tokens_from_string(str(messages), "gpt-3.5-turbo")

getTokens(prompt1, doc, query)

175

In [107]:
import pandas as pd
dataframe1 = pd.read_excel('dataset_gpt35turbo.xlsx')
dataframe1

,id,Type,query,reference_contexts,reference_answer,reference_answer_by,query_by
0,0,Basic,What were some of the key industries that drov...,['This is the main article of a series that co...,Some of the key industries that drove the earl...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo)
1,1,Basic,Describe the early settlement and founding of ...,['This is the main article of a series that co...,The early settlement and founding of Seattle i...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo)
2,2,Basic,What role did Arthur A. Denny play in the deve...,"[""He eventually gave a land grant to the Unive...",Arthur A. Denny gave a land grant to the Unive...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo)
3,3,Basic,Describe the establishment of Seattle's first ...,"[""He eventually gave a land grant to the Unive...",Francis X. Prefontaine arrived in Seattle in 1...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo)
4,4,Basic,How did Fr. Prefontaine raise funds to build S...,"['At that time, Seattle had no Catholic church...",Fr. Prefontaine raised funds to build Seattle'...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo)
...,...,...,...,...,...,...,...
284,284,No meaningfull question/result,Pizza pizza pizza,['=== Reform War and Second Empire ===\n\nPeac...,I am sorry I cannot answer your question,human,human
285,285,No meaningfull question/result,Nah,['=== Porfiriato (1876–1910) ===\n\nPresident ...,I am sorry I cannot answer your question,human,human
286,286,No meaningfull question/result,,['=== Porfiriato (1876–1910) ===\n\nPresident ...,I am sorry I cannot answer your question,human,human
287,287,No meaningfull question/result,Describe the events of La decena trágica and i...,[],I am sorry I cannot answer your question,human,human


In [108]:
prompt1 = open('Prompt1.txt', 'r').read()
dataframe1["Output_v1_0"] = dataframe1.apply(lambda x: getResponse(prompt1,x["reference_contexts"],x["query"]),axis = 1)
dataframe1

,id,Type,query,reference_contexts,reference_answer,reference_answer_by,query_by,Output_v1_0
0,0,Basic,What were some of the key industries that drov...,['This is the main article of a series that co...,Some of the key industries that drove the earl...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo),response:\nSome of the key industries that dro...
1,1,Basic,Describe the early settlement and founding of ...,['This is the main article of a series that co...,The early settlement and founding of Seattle i...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo),response:\nSeattle was founded in the mid-19th...
2,2,Basic,What role did Arthur A. Denny play in the deve...,"[""He eventually gave a land grant to the Unive...",Arthur A. Denny gave a land grant to the Unive...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo),response:\nArthur A. Denny gave a land grant t...
3,3,Basic,Describe the establishment of Seattle's first ...,"[""He eventually gave a land grant to the Unive...",Francis X. Prefontaine arrived in Seattle in 1...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo),"response:\nIn 1867, a young French Canadian Ca..."
4,4,Basic,How did Fr. Prefontaine raise funds to build S...,"['At that time, Seattle had no Catholic church...",Fr. Prefontaine raised funds to build Seattle'...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo),response:\nFr. Prefontaine raised the money to...
...,...,...,...,...,...,...,...,...
284,284,No meaningfull question/result,Pizza pizza pizza,['=== Reform War and Second Empire ===\n\nPeac...,I am sorry I cannot answer your question,human,human,I am sorry I cannot answer your question.
285,285,No meaningfull question/result,Nah,['=== Porfiriato (1876–1910) ===\n\nPresident ...,I am sorry I cannot answer your question,human,human,I am sorry I cannot answer your question.
286,286,No meaningfull question/result,,['=== Porfiriato (1876–1910) ===\n\nPresident ...,I am sorry I cannot answer your question,human,human,I am sorry I cannot answer your question.
287,287,No meaningfull question/result,Describe the events of La decena trágica and i...,[],I am sorry I cannot answer your question,human,human,I am sorry I cannot answer your question.


In [114]:
dataframe1["BLEU"] = dataframe1.apply(lambda x: bleu(x["reference_answer"],x["Output_v1_0"].replace('response:\n', '')),axis = 1)
dataframe1["ROUGE"] = dataframe1.apply(lambda x: RougeScore(x["reference_answer"],x["Output_v1_0"].replace('response:\n', '')),axis = 1)
print("BLEU AVG = {}".format(dataframe1["BLEU"].mean()))
print("ROUGE AVG = {}".format(dataframe1["ROUGE"].mean()))

C:\Users\edcastr\AppData\Local\anaconda3\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\edcastr\AppData\Local\anaconda3\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\edcastr\AppData\Local\anaconda3\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower 

BLEU AVG = 9.986672752529635e-232
ROUGE AVG = 0.6180562184893251


In [136]:
dataframe1["Tokens"] = dataframe1.apply(lambda x: getTokens(prompt1,x["reference_contexts"],x["query"]),axis = 1)
print("Avg tokens: {}".format(dataframe1["Tokens"].mean()))

Avg tokens: 1096.235294117647


In [137]:
print("max tokens: {}".format(dataframe1["Tokens"].max()))

max tokens: 2253


In [138]:
print("min tokens: {}".format(dataframe1["Tokens"].min()))

min tokens: 160


In [139]:
dataframe1.to_excel("output1.xlsx")

In [172]:
dataframe1 = pd.read_excel('dataset_gpt35turbo.xlsx')
prompt2 = open('StructuredPrompt.txt', 'r').read()
dataframe1["Output_v1_0"] = dataframe1.apply(lambda x: getResponse(prompt2,x["reference_contexts"],x["query"]),axis = 1)
dataframe1

,id,Type,query,reference_contexts,reference_answer,reference_answer_by,query_by,reference_citation,Output_v1_0
0,0,Basic,What were some of the key industries that drov...,['This is the main article of a series that co...,Some of the key industries that drove the earl...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo),"[""document index 0""]","{\n\t""_original_query"": ""What were some of the..."
1,1,Basic,Describe the early settlement and founding of ...,['This is the main article of a series that co...,The early settlement and founding of Seattle i...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo),"[""document index 0""]","{\n\t""_original_query"": ""Describe the early se..."
2,2,Basic,What role did Arthur A. Denny play in the deve...,"[""He eventually gave a land grant to the Unive...",Arthur A. Denny gave a land grant to the Unive...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo),"[""document index 0""]","{\n\t""_original_query"": ""What role did Arthur ..."
3,3,Basic,Describe the establishment of Seattle's first ...,"[""He eventually gave a land grant to the Unive...",Francis X. Prefontaine arrived in Seattle in 1...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo),"[""document index 0""]","{\n\t""_original_query"": ""Describe the establis..."
4,4,Basic,How did Fr. Prefontaine raise funds to build S...,"['At that time, Seattle had no Catholic church...",Fr. Prefontaine raised funds to build Seattle'...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo),"[""document index 0""]","{\n\t""_original_query"": ""How did Fr. Prefontai..."
...,...,...,...,...,...,...,...,...,...
284,284,No meaningfull question/result,Pizza pizza pizza,['=== Reform War and Second Empire ===\n\nPeac...,I am sorry I cannot answer your question,human,human,[],"{\n\t""_original_query"": ""Pizza pizza pizza"",\n..."
285,285,No meaningfull question/result,Nah,['=== Porfiriato (1876–1910) ===\n\nPresident ...,I am sorry I cannot answer your question,human,human,[],"{\n\t""_original_query"": ""Nah"",\n\t""_reasoning""..."
286,286,No meaningfull question/result,,['=== Porfiriato (1876–1910) ===\n\nPresident ...,I am sorry I cannot answer your question,human,human,[],"{\n\t""_original_query"": ' ',\n\t""_reasoning""..."
287,287,No meaningfull question/result,Describe the events of La decena trágica and i...,[],I am sorry I cannot answer your question,human,human,[],"{\n\t""_original_query"": ""Describe the events o..."


In [173]:
import json

def getResult(output):
    try:
        return json.loads(output)["response"]
    except:
        return output

getResult('{ "_original_query": " ", "_reasoning":{"_Explanation": " ", "_Decision": " ", "_Safety_assessment": " " }, "response": "I am sorry I cannot answer your question", "citation": [] }')

'I am sorry I cannot answer your question'

In [174]:
dataframe1["response"] = dataframe1.apply(lambda x: getResult(x["Output_v1_0"]),axis = 1)
dataframe1

,id,Type,query,reference_contexts,reference_answer,reference_answer_by,query_by,reference_citation,Output_v1_0,response
0,0,Basic,What were some of the key industries that drov...,['This is the main article of a series that co...,Some of the key industries that drove the earl...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo),"[""document index 0""]","{\n\t""_original_query"": ""What were some of the...",The key industries that drove the early develo...
1,1,Basic,Describe the early settlement and founding of ...,['This is the main article of a series that co...,The early settlement and founding of Seattle i...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo),"[""document index 0""]","{\n\t""_original_query"": ""Describe the early se...",Seattle was founded with the arrival of the De...
2,2,Basic,What role did Arthur A. Denny play in the deve...,"[""He eventually gave a land grant to the Unive...",Arthur A. Denny gave a land grant to the Unive...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo),"[""document index 0""]","{\n\t""_original_query"": ""What role did Arthur ...",Arthur A. Denny played a key role in the devel...
3,3,Basic,Describe the establishment of Seattle's first ...,"[""He eventually gave a land grant to the Unive...",Francis X. Prefontaine arrived in Seattle in 1...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo),"[""document index 0""]","{\n\t""_original_query"": ""Describe the establis...",Francis X. Prefontaine faced the challenge of ...
4,4,Basic,How did Fr. Prefontaine raise funds to build S...,"['At that time, Seattle had no Catholic church...",Fr. Prefontaine raised funds to build Seattle'...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo),"[""document index 0""]","{\n\t""_original_query"": ""How did Fr. Prefontai...",Fr. Prefontaine raised the money by holding fa...
...,...,...,...,...,...,...,...,...,...,...
284,284,No meaningfull question/result,Pizza pizza pizza,['=== Reform War and Second Empire ===\n\nPeac...,I am sorry I cannot answer your question,human,human,[],"{\n\t""_original_query"": ""Pizza pizza pizza"",\n...",I am sorry I cannot answer your question
285,285,No meaningfull question/result,Nah,['=== Porfiriato (1876–1910) ===\n\nPresident ...,I am sorry I cannot answer your question,human,human,[],"{\n\t""_original_query"": ""Nah"",\n\t""_reasoning""...",I am sorry I cannot answer your question
286,286,No meaningfull question/result,,['=== Porfiriato (1876–1910) ===\n\nPresident ...,I am sorry I cannot answer your question,human,human,[],"{\n\t""_original_query"": ' ',\n\t""_reasoning""...","{\n\t""_original_query"": ' ',\n\t""_reasoning""..."
287,287,No meaningfull question/result,Describe the events of La decena trágica and i...,[],I am sorry I cannot answer your question,human,human,[],"{\n\t""_original_query"": ""Describe the events o...",I am sorry I cannot answer your question


In [175]:
#dataframe1["BLEU"] = dataframe1.apply(lambda x: bleu(x["reference_answer"],x["Output_v1_0"].replace('response:\n', '')),axis = 1)
dataframe1["ROUGE"] = dataframe1.apply(lambda x: RougeScore(x["reference_answer"],x["response"]),axis = 1)
#print("BLEU AVG = {}".format(dataframe1["BLEU"].mean()))
print("ROUGE AVG = {}".format(dataframe1["ROUGE"].mean()))

ROUGE AVG = 0.5966760432156795


In [169]:
dataframe1["Tokens"] = dataframe1.apply(lambda x: getTokens(prompt2,x["reference_contexts"],x["query"]),axis = 1)
print("Avg tokens: {}".format(dataframe1["Tokens"].mean()))

Avg tokens: 3462.740484429066


In [176]:
dataframe1.to_excel("output2.xlsx")